In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier

In [6]:
df =pd.read_csv ("lung_cancer_mortality_data_large_v2.csv")

df.head()

,id,age,gender,country,diagnosis_date,cancer_stage,beginning_of_treatment_date,family_history,smoking_status,bmi,cholesterol_level,hypertension,asthma,cirrhosis,other_cancer,treatment_type,end_treatment_date,survived
0,1,64.0,Female,Germany,2016-04-07,Stage III,2016-04-21,No,Never Smoked,31.1,257,1,1,0,0,Combined,2017-11-15,0
1,2,50.0,Male,Czech Republic,2023-04-22,Stage III,2023-05-02,Yes,Passive Smoker,25.9,208,1,0,0,0,Radiation,2024-04-25,0
2,3,65.0,Male,Romania,2023-04-07,Stage IV,2023-04-12,No,Never Smoked,18.9,193,0,0,0,0,Surgery,2025-03-11,0
3,4,51.0,Female,Latvia,2016-02-07,Stage III,2016-02-13,Yes,Former Smoker,34.6,249,1,1,1,0,Surgery,2017-04-14,1
4,5,37.0,Male,Greece,2023-12-01,Stage I,2023-12-03,Yes,Never Smoked,40.2,262,0,0,0,0,Chemotherapy,2024-09-20,0


## 1. **Pais del cual provienen la mayor cantidad de pacientes**

In [7]:

# # Contar cuántos pacientes hay por país
conteo_paises = df['country'].value_counts()

# # Obtener el país con más pacientes
pais_principal = conteo_paises.idxmax()
cantidad = conteo_paises.max()

print(f"\nEl país con más pacientes es: {pais_principal} ")



El país con más pacientes es: Latvia 


## 2. **Cuantos sobrevivieron y cuantos no**

In [8]:
# Contar cuántos sobrevivieron (1) y cuántos no (0)
conteo_supervivencia = df['survived'].value_counts()

sobrevivieron = conteo_supervivencia.get(1, 0)
no_sobrevivieron = conteo_supervivencia.get(0, 0)

print(f"\nSobrevivieron: {sobrevivieron}")
print(f"No sobrevivieron: {no_sobrevivieron}")


Sobrevivieron: 713702
No sobrevivieron: 2536298


### 3 -Relacion entre genero y supervivencia

In [9]:
# Codificar gender
df['gender_code'] = df['gender'].map({'Female': 0, 'Male': 1})

# Correlación de Pearson entre género codificado y supervivencia
correlacion = df['gender_code'].corr(df['survived'])

print(f"Correlación (Pearson) entre género y supervivencia: {correlacion:.2f}")


Correlación (Pearson) entre género y supervivencia: -0.00


In [10]:
# Tabla de contingencia entre género y supervivencia
tabla_genero_supervivencia = pd.crosstab(df['gender'], df['survived'])

print(tabla_genero_supervivencia)


survived        0       1
gender                   
Female    1268524  356966
Male      1267774  356736


## 4. ¿Existe alguna diferencia notable en la edad de los pacientes que sobrevivieron y los que no?

In [11]:
# Edad media según supervivencia
media_edades = df.groupby('survived')['age'].mean()
print(media_edades)

survived
0    54.993239
1    55.022637
Name: age, dtype: float64


0     # No sobrevivieron
1     # Sobrevivieron
---
0    54.993239
1    55.022637
Name: age, dtype: float64

###  5. **¿Que estadio de cancer tiene la mayor tasa de supervivencia?**

In [12]:
# Calcular tasa de supervivencia por estadio
tasa_supervivencia = df.groupby('cancer_stage')['survived'].mean().sort_values(ascending=False)
print(tasa_supervivencia)

cancer_stage
Stage IV     0.219906
Stage I      0.219700
Stage III    0.219535
Stage II     0.219261
Name: survived, dtype: float64


In [13]:
### ✅ 3. **¿Qué estadio tiene la mayor tasa?**

mejor_estadio = tasa_supervivencia.idxmax()
print(f"El estadio con mayor tasa de supervivencia es: {mejor_estadio}")

El estadio con mayor tasa de supervivencia es: Stage IV


## 6. **¿Que impacto tiene el indice de masa corporal (bmi) en la supervivencia?**

In [14]:
df.groupby('survived')['bmi'].mean()

# En promedio, hay diferencia en el BMI entre los pacientes que sobrevivieron y los que no.

survived
0    30.492094
1    30.495128
Name: bmi, dtype: float64

In [15]:
correlacion = df['bmi'].corr(df['survived'])
print(f"Correlación entre BMI y supervivencia: {correlacion:.2f}")

Correlación entre BMI y supervivencia: 0.00


 **Interpretación**:

* Valor cercano a **0**: poca o ninguna relación.
* Valor positivo: a mayor BMI, mayor probabilidad de supervivencia.
* Valor negativo: a mayor BMI, menor probabilidad.